<a href="https://colab.research.google.com/github/yms0606/SecuLayer/blob/main/%EC%9B%94%EA%B0%84%20%EB%8D%B0%EC%9D%B4%EC%BD%98%20%EC%BD%94%EB%93%9C%20%EC%9C%A0%EC%82%AC%EC%84%B1%20%ED%8C%90%EB%8B%A8%20AI%20%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C/%EC%9B%94%EA%B0%84_%EB%8D%B0%EC%9D%B4%EC%BD%98_%EC%BD%94%EB%93%9C_%EC%9C%A0%EC%82%AC%EC%84%B1_%ED%8C%90%EB%8B%A8_AI_%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U transformers sentence-transformers

In [2]:
import pandas as pd
import numpy as np

In [3]:
train = pd.read_csv('/content/drive/MyDrive/ML study/code_sim/sample_train.csv')
test = pd.read_csv('/content/drive/MyDrive/ML study/code_sim/test.csv')
submission = pd.read_csv('/content/drive/MyDrive/ML study/code_sim/sample_submission.csv')

In [ ]:
train.head()

In [4]:
from sklearn.model_selection import train_test_split

train, valid = train_test_split(train, test_size=0.2, random_state=42, stratify = train['similar'])

In [5]:
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers import SentenceTransformer, InputExample, losses
from sentence_transformers.cross_encoder.evaluation import CEBinaryClassificationEvaluator
from torch.utils.data import DataLoader

In [6]:
model = CrossEncoder('cross-encoder/stsb-roberta-base',num_labels=1)

In [7]:
from tqdm import tqdm
train_examples = []
valid_examples = []

for idx, row in tqdm(train.iterrows()):
  train_examples.append(InputExample(texts=[row['code1'], row['code2']], label=row['similar']))

for idx, row in tqdm(valid.iterrows()):
  valid_examples.append(InputExample(texts=[row['code1'], row['code2']], label=row['similar']))

14376it [00:00, 21633.99it/s]
3594it [00:00, 21817.19it/s]


In [8]:
train_dataloader = DataLoader(train_examples, shuffle = True, batch_size = 16)
valid_dataloader = CEBinaryClassificationEvaluator.from_input_examples(valid_examples)

In [9]:
model.fit(train_dataloader=train_dataloader, evaluator = valid_dataloader, epochs=1, warmup_steps=100)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/899 [00:00<?, ?it/s]

In [10]:
test_examples = []
for idx, row in tqdm(test.iterrows()):
  test_examples.append([row['code1'],row['code2']])

LEN_TEST = len(test_examples)

preds = []
for idx in tqdm(range(LEN_TEST)):
  preds.append(model.predict(test_examples[idx]))

submission['similar'] = preds
submission['similar'] = submission['similar'].apply(lambda x:0 if x<0.5 else 1)

submission.to_csv('sub_code.csv',index=False)

179700it [00:07, 23911.58it/s]
100%|██████████| 179700/179700 [1:18:26<00:00, 38.18it/s]
